In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize

In [14]:
# Load the Excel file into a pandas DataFrame
df = pd.read_excel('/content/dataset_v2.xlsx')

# Display the first few rows of the DataFrame
display(df)

,id_review,review,sentiment,rating,labels,is_labels_validated,model_version,review_date
0,00000038-3b3b-5102-be9a-7cf800000000,Nell’applicazione (ricarica conto)\nnon ho più...,Negative,5.0,"[""problemi_ricarica""]",0.0,1.0.0,2024-10-01
1,00000038-3b3b-5102-bed4-a60f00000000,mi succede svariate volte che non riesco ad ac...,Negative,1.0,"[""problemi_accesso""]",0.0,1.0.0,2024-10-02
2,00000038-3b3b-5102-bed5-edb100000000,"Applicazione troppo lenta, non si riesce a far...",Negative,1.0,"[""problemi_sistema""]",0.0,1.0.0,2024-10-02
3,00000038-3b3b-5102-bf2b-621400000000,"ho hype da due anni, mi è arrivata la nuova ca...",Negative,4.0,"[""problemi_gestione_carta""]",0.0,1.0.0,2024-10-04
4,00000038-3b3b-5102-c000-0fc900000000,Da oggi sto riscontrando problemi ad accedere ...,Negative,1.0,"[""problemi_accesso"",""problemi_assistenza""]",0.0,1.0.0,2024-10-08
...,...,...,...,...,...,...,...,...
6145,NaN,è una vergogna pagare 3 euro al mese e avere p...,negative,NaN,"[""problemi_bonifici"",""problemi_di_accesso""]",NaN,1.0.0,NaT
6146,NaN,“Accesso biometrico non valido “ comincia a da...,negative,NaN,"[""problemi_di_accesso""]",NaN,1.0.0,NaT
6147,NaN,“Accesso biometrico non valido” è un messaggio...,negative,NaN,"[""problemi_di_accesso""]",NaN,1.0.0,NaT
6148,NaN,“Funzione” che richiedo da tempo: una sezione ...,negative,NaN,"[""problemi_di_comunicazione""]",NaN,1.0.0,NaT


In [15]:
df['sentiment'] = df['sentiment'].astype(str).str.lower()
df['sentiment'].unique()

array(['negative', 'positive', 'neutral'], dtype=object)

In [16]:
dfNeu=df[df['sentiment']=='neutral']

In [17]:
display(dfNeu)

,id_review,review,sentiment,rating,labels,is_labels_validated,model_version,review_date
6,00000038-3b3b-5102-c02b-ca6f00000000,"App perfetta, nessun altra banca ha questa imm...",neutral,4.0,"[""problemi_gestione_carta""]",0.0,1.0.0,2024-10-09
10,00000038-3b3b-5102-c160-dcfb00000000,Buona app a volte non fluido l’accesso. App or...,neutral,3.0,"[""problemi_accesso""]",0.0,1.0.0,2024-10-14
15,00000038-3b3b-5102-c21a-7bad00000000,Appena aperto il conto Premium e spero di trov...,neutral,4.0,"[""problemi_comunicazione""]",0.0,1.0.0,2024-10-18
18,00000038-3b3b-5102-c341-72bb00000000,"Salve, la funzionalità di tracciamento delle o...",neutral,1.0,"[""problemi_pagamenti_generici""]",0.0,1.0.0,2024-10-23
28,00000038-3b3b-5102-c7c2-0ab000000000,Apro l’app più volte a settimana e ogni volta ...,neutral,1.0,"[""problemi_accesso""]",0.0,1.0.0,2024-11-12
...,...,...,...,...,...,...,...,...
3672,f408d06f-3a29-4c4f-85c4-d1231eafef9e,Correggo la valutazione data in precedenza per...,neutral,5.0,"[""problemi_sistema""]",0.0,1.0.0,2024-11-16
3682,f5a79382-7a09-463d-83d6-bedc2668640b,Improvvisamente da ieri il mio saldo non è qua...,neutral,5.0,"[""problemi_pagamenti_generici""]",0.0,1.0.0,2024-11-19
3687,f68f611e-6380-4e6e-a2fc-68b98847aba3,Dopo l'ultimo aggiornamento è sparito il ricon...,neutral,2.0,"[""problemi_accesso""]",0.0,1.0.0,2025-01-19
3696,f886e36a-faaf-423a-bc67-767174fcf788,Ottima App. Migliorerei la reportistica che fa...,neutral,5.0,"[""problemi_comunicazione""]",0.0,1.0.0,2024-10-16


In [18]:
#As input we will have all italian reviews...to test let's detect language and discard non italian ones

In [19]:
!pip install langdetect

In [20]:
from langdetect import detect, DetectorFactory
# Set seed for reproducibility
DetectorFactory.seed = 0

# Define a function to safely detect language
def safe_detect_language(text):
    try:
        return detect(str(text))
    except:
        return np.nan

# Apply the function to the 'review' column
dfNeu['language'] = dfNeu['review'].apply(safe_detect_language)


/tmp/ipython-input-1627473701.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfNeu['language'] = dfNeu['review'].apply(safe_detect_language)


In [21]:
dfNeu=dfNeu[dfNeu['language']=='it']
display(dfNeu)

,id_review,review,sentiment,rating,labels,is_labels_validated,model_version,review_date,language
6,00000038-3b3b-5102-c02b-ca6f00000000,"App perfetta, nessun altra banca ha questa imm...",neutral,4.0,"[""problemi_gestione_carta""]",0.0,1.0.0,2024-10-09,it
10,00000038-3b3b-5102-c160-dcfb00000000,Buona app a volte non fluido l’accesso. App or...,neutral,3.0,"[""problemi_accesso""]",0.0,1.0.0,2024-10-14,it
15,00000038-3b3b-5102-c21a-7bad00000000,Appena aperto il conto Premium e spero di trov...,neutral,4.0,"[""problemi_comunicazione""]",0.0,1.0.0,2024-10-18,it
18,00000038-3b3b-5102-c341-72bb00000000,"Salve, la funzionalità di tracciamento delle o...",neutral,1.0,"[""problemi_pagamenti_generici""]",0.0,1.0.0,2024-10-23,it
28,00000038-3b3b-5102-c7c2-0ab000000000,Apro l’app più volte a settimana e ogni volta ...,neutral,1.0,"[""problemi_accesso""]",0.0,1.0.0,2024-11-12,it
...,...,...,...,...,...,...,...,...,...
3672,f408d06f-3a29-4c4f-85c4-d1231eafef9e,Correggo la valutazione data in precedenza per...,neutral,5.0,"[""problemi_sistema""]",0.0,1.0.0,2024-11-16,it
3682,f5a79382-7a09-463d-83d6-bedc2668640b,Improvvisamente da ieri il mio saldo non è qua...,neutral,5.0,"[""problemi_pagamenti_generici""]",0.0,1.0.0,2024-11-19,it
3687,f68f611e-6380-4e6e-a2fc-68b98847aba3,Dopo l'ultimo aggiornamento è sparito il ricon...,neutral,2.0,"[""problemi_accesso""]",0.0,1.0.0,2025-01-19,it
3696,f886e36a-faaf-423a-bc67-767174fcf788,Ottima App. Migliorerei la reportistica che fa...,neutral,5.0,"[""problemi_comunicazione""]",0.0,1.0.0,2024-10-16,it


In [22]:
!pip install emoji

In [23]:
import re
import emoji
from langdetect import detect


def clean_text(text):

    # normalize spacing
    text = text.strip()
    text = re.sub(r"\s+", " ", text)

    #replace URLs, mentions
    text = re.sub(r"http\S+", "<URL>", text)
    text = re.sub(r"@\w+", "<USER>", text)

    # reduce repeated chars (bello → bello, belloooo → bello)
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)

    return text


def preprocess_for_transformer(text):
    cleaned = clean_text(text)

    encoded = tokenizer(
        cleaned,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    return encoded


In [24]:
dfNeu['cleaned_review'] = dfNeu['review'].apply(clean_text)
display(dfNeu)

,id_review,review,sentiment,rating,labels,is_labels_validated,model_version,review_date,language,cleaned_review
6,00000038-3b3b-5102-c02b-ca6f00000000,"App perfetta, nessun altra banca ha questa imm...",neutral,4.0,"[""problemi_gestione_carta""]",0.0,1.0.0,2024-10-09,it,"App perfetta, nessun altra banca ha questa imm..."
10,00000038-3b3b-5102-c160-dcfb00000000,Buona app a volte non fluido l’accesso. App or...,neutral,3.0,"[""problemi_accesso""]",0.0,1.0.0,2024-10-14,it,Buona app a volte non fluido l’accesso. App or...
15,00000038-3b3b-5102-c21a-7bad00000000,Appena aperto il conto Premium e spero di trov...,neutral,4.0,"[""problemi_comunicazione""]",0.0,1.0.0,2024-10-18,it,Appena aperto il conto Premium e spero di trov...
18,00000038-3b3b-5102-c341-72bb00000000,"Salve, la funzionalità di tracciamento delle o...",neutral,1.0,"[""problemi_pagamenti_generici""]",0.0,1.0.0,2024-10-23,it,"Salve, la funzionalità di tracciamento delle o..."
28,00000038-3b3b-5102-c7c2-0ab000000000,Apro l’app più volte a settimana e ogni volta ...,neutral,1.0,"[""problemi_accesso""]",0.0,1.0.0,2024-11-12,it,Apro l’app più volte a settimana e ogni volta ...
...,...,...,...,...,...,...,...,...,...,...
3672,f408d06f-3a29-4c4f-85c4-d1231eafef9e,Correggo la valutazione data in precedenza per...,neutral,5.0,"[""problemi_sistema""]",0.0,1.0.0,2024-11-16,it,Correggo la valutazione data in precedenza per...
3682,f5a79382-7a09-463d-83d6-bedc2668640b,Improvvisamente da ieri il mio saldo non è qua...,neutral,5.0,"[""problemi_pagamenti_generici""]",0.0,1.0.0,2024-11-19,it,Improvvisamente da ieri il mio saldo non è qua...
3687,f68f611e-6380-4e6e-a2fc-68b98847aba3,Dopo l'ultimo aggiornamento è sparito il ricon...,neutral,2.0,"[""problemi_accesso""]",0.0,1.0.0,2025-01-19,it,Dopo l'ultimo aggiornamento è sparito il ricon...
3696,f886e36a-faaf-423a-bc67-767174fcf788,Ottima App. Migliorerei la reportistica che fa...,neutral,5.0,"[""problemi_comunicazione""]",0.0,1.0.0,2024-10-16,it,Ottima App. Migliorerei la reportistica che fa...


In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "MilaNLProc/feel-it-italian-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


In [26]:
import re
import emoji
import torch

#-----------------
# 2. Clean Function
# -------------------------
def clean_text(text):

    # Normalize spacing
    text = text.strip()
    text = re.sub(r"\s+", " ", text)

    # Replace URLs and @mentions
    text = re.sub(r"http\S+", "<URL>", text)
    text = re.sub(r"@\w+", "<USER>", text)

    # Normalize repeated characters (belloooo → belloo)
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)

    # Convert emojis to textual tags
    text = emoji.replace_emoji(text, replace=lambda e, data: f" {emoji.demojize(e)} ")

    return text


# -------------------------
# 3. Sentiment Prediction (negative / neutral / positive)
# -------------------------
def predict_sentiment(text):

    cleaned = clean_text(text)

    # Tokenization
    enc = tokenizer(
        cleaned,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Model inference
    with torch.no_grad():
        logits = model(**enc).logits
        pred = torch.argmax(logits, dim=1).item()

    # FEEL-IT labels
    labels = ["negative", "neutral", "positive"]
    return labels[pred]



In [27]:
dfNeu["sentiment_pred"] = dfNeu["review"].apply(predict_sentiment)

In [28]:
display(dfNeu)

,id_review,review,sentiment,rating,labels,is_labels_validated,model_version,review_date,language,cleaned_review,sentiment_pred
6,00000038-3b3b-5102-c02b-ca6f00000000,"App perfetta, nessun altra banca ha questa imm...",neutral,4.0,"[""problemi_gestione_carta""]",0.0,1.0.0,2024-10-09,it,"App perfetta, nessun altra banca ha questa imm...",negative
10,00000038-3b3b-5102-c160-dcfb00000000,Buona app a volte non fluido l’accesso. App or...,neutral,3.0,"[""problemi_accesso""]",0.0,1.0.0,2024-10-14,it,Buona app a volte non fluido l’accesso. App or...,negative
15,00000038-3b3b-5102-c21a-7bad00000000,Appena aperto il conto Premium e spero di trov...,neutral,4.0,"[""problemi_comunicazione""]",0.0,1.0.0,2024-10-18,it,Appena aperto il conto Premium e spero di trov...,neutral
18,00000038-3b3b-5102-c341-72bb00000000,"Salve, la funzionalità di tracciamento delle o...",neutral,1.0,"[""problemi_pagamenti_generici""]",0.0,1.0.0,2024-10-23,it,"Salve, la funzionalità di tracciamento delle o...",negative
28,00000038-3b3b-5102-c7c2-0ab000000000,Apro l’app più volte a settimana e ogni volta ...,neutral,1.0,"[""problemi_accesso""]",0.0,1.0.0,2024-11-12,it,Apro l’app più volte a settimana e ogni volta ...,negative
...,...,...,...,...,...,...,...,...,...,...,...
3672,f408d06f-3a29-4c4f-85c4-d1231eafef9e,Correggo la valutazione data in precedenza per...,neutral,5.0,"[""problemi_sistema""]",0.0,1.0.0,2024-11-16,it,Correggo la valutazione data in precedenza per...,negative
3682,f5a79382-7a09-463d-83d6-bedc2668640b,Improvvisamente da ieri il mio saldo non è qua...,neutral,5.0,"[""problemi_pagamenti_generici""]",0.0,1.0.0,2024-11-19,it,Improvvisamente da ieri il mio saldo non è qua...,negative
3687,f68f611e-6380-4e6e-a2fc-68b98847aba3,Dopo l'ultimo aggiornamento è sparito il ricon...,neutral,2.0,"[""problemi_accesso""]",0.0,1.0.0,2025-01-19,it,Dopo l'ultimo aggiornamento è sparito il ricon...,negative
3696,f886e36a-faaf-423a-bc67-767174fcf788,Ottima App. Migliorerei la reportistica che fa...,neutral,5.0,"[""problemi_comunicazione""]",0.0,1.0.0,2024-10-16,it,Ottima App. Migliorerei la reportistica che fa...,neutral


In [29]:
import re
import emoji
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load multilingual sentiment model (XLM-R)
model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

# Cleaning function
def clean_text(text):

    text = text.strip()
    text = re.sub(r"\s+", " ", text)

    # Replace URLs and mentions
    text = re.sub(r"http\S+", "<URL>", text)
    text = re.sub(r"@\w+", "<USER>", text)

    # Reduce repeated characters
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)

    # Emoji → textual representation
    text = emoji.replace_emoji(text, replace=lambda e, data: f" {emoji.demojize(e)} ")

    return text





pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [30]:

# Predict sentiment (negative / neutral / positive)
def predict_sentimentMult(text):

    cleaned = clean_text(text)

    # Tokenization
    enc = tokenizer(
        cleaned,
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    with torch.no_grad():
        logits = model(**enc).logits
        pred = torch.argmax(logits, dim=1).item()

    labels = ["negative", "neutral", "positive"]
    return labels[pred]


In [31]:
dfNeu["sentiment_predMulti"] = dfNeu["review"].apply(predict_sentimentMult)

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [32]:
display(dfNeu)

,id_review,review,sentiment,rating,labels,is_labels_validated,model_version,review_date,language,cleaned_review,sentiment_pred,sentiment_predMulti
6,00000038-3b3b-5102-c02b-ca6f00000000,"App perfetta, nessun altra banca ha questa imm...",neutral,4.0,"[""problemi_gestione_carta""]",0.0,1.0.0,2024-10-09,it,"App perfetta, nessun altra banca ha questa imm...",negative,negative
10,00000038-3b3b-5102-c160-dcfb00000000,Buona app a volte non fluido l’accesso. App or...,neutral,3.0,"[""problemi_accesso""]",0.0,1.0.0,2024-10-14,it,Buona app a volte non fluido l’accesso. App or...,negative,positive
15,00000038-3b3b-5102-c21a-7bad00000000,Appena aperto il conto Premium e spero di trov...,neutral,4.0,"[""problemi_comunicazione""]",0.0,1.0.0,2024-10-18,it,Appena aperto il conto Premium e spero di trov...,neutral,positive
18,00000038-3b3b-5102-c341-72bb00000000,"Salve, la funzionalità di tracciamento delle o...",neutral,1.0,"[""problemi_pagamenti_generici""]",0.0,1.0.0,2024-10-23,it,"Salve, la funzionalità di tracciamento delle o...",negative,negative
28,00000038-3b3b-5102-c7c2-0ab000000000,Apro l’app più volte a settimana e ogni volta ...,neutral,1.0,"[""problemi_accesso""]",0.0,1.0.0,2024-11-12,it,Apro l’app più volte a settimana e ogni volta ...,negative,negative
...,...,...,...,...,...,...,...,...,...,...,...,...
3672,f408d06f-3a29-4c4f-85c4-d1231eafef9e,Correggo la valutazione data in precedenza per...,neutral,5.0,"[""problemi_sistema""]",0.0,1.0.0,2024-11-16,it,Correggo la valutazione data in precedenza per...,negative,negative
3682,f5a79382-7a09-463d-83d6-bedc2668640b,Improvvisamente da ieri il mio saldo non è qua...,neutral,5.0,"[""problemi_pagamenti_generici""]",0.0,1.0.0,2024-11-19,it,Improvvisamente da ieri il mio saldo non è qua...,negative,negative
3687,f68f611e-6380-4e6e-a2fc-68b98847aba3,Dopo l'ultimo aggiornamento è sparito il ricon...,neutral,2.0,"[""problemi_accesso""]",0.0,1.0.0,2025-01-19,it,Dopo l'ultimo aggiornamento è sparito il ricon...,negative,negative
3696,f886e36a-faaf-423a-bc67-767174fcf788,Ottima App. Migliorerei la reportistica che fa...,neutral,5.0,"[""problemi_comunicazione""]",0.0,1.0.0,2024-10-16,it,Ottima App. Migliorerei la reportistica che fa...,neutral,positive


In [33]:
from transformers import pipeline

classifier = pipeline(
    "text-classification",
    model="MilaNLProc/feel-it-italian-sentiment",
    top_k=None
)


Device set to use cpu


In [34]:
dfNeu["sentiment_predScore"] = dfNeu["review"].apply(classifier)


In [35]:
dfNeu["sentiment_predScoreNumber"] =dfNeu["sentiment_predScore"].apply(lambda x: x[0][0]["score"])
dfNeu["sentiment_predScoreLabel"] =dfNeu["sentiment_predScore"].apply(lambda x: x[0][0]["label"])

In [36]:
display(dfNeu)

,id_review,review,sentiment,rating,labels,is_labels_validated,model_version,review_date,language,cleaned_review,sentiment_pred,sentiment_predMulti,sentiment_predScore,sentiment_predScoreNumber,sentiment_predScoreLabel
6,00000038-3b3b-5102-c02b-ca6f00000000,"App perfetta, nessun altra banca ha questa imm...",neutral,4.0,"[""problemi_gestione_carta""]",0.0,1.0.0,2024-10-09,it,"App perfetta, nessun altra banca ha questa imm...",negative,negative,"[[{'label': 'negative', 'score': 0.99960654973...",0.999607,negative
10,00000038-3b3b-5102-c160-dcfb00000000,Buona app a volte non fluido l’accesso. App or...,neutral,3.0,"[""problemi_accesso""]",0.0,1.0.0,2024-10-14,it,Buona app a volte non fluido l’accesso. App or...,negative,positive,"[[{'label': 'negative', 'score': 0.99854135513...",0.998541,negative
15,00000038-3b3b-5102-c21a-7bad00000000,Appena aperto il conto Premium e spero di trov...,neutral,4.0,"[""problemi_comunicazione""]",0.0,1.0.0,2024-10-18,it,Appena aperto il conto Premium e spero di trov...,neutral,positive,"[[{'label': 'positive', 'score': 0.99924963712...",0.999250,positive
18,00000038-3b3b-5102-c341-72bb00000000,"Salve, la funzionalità di tracciamento delle o...",neutral,1.0,"[""problemi_pagamenti_generici""]",0.0,1.0.0,2024-10-23,it,"Salve, la funzionalità di tracciamento delle o...",negative,negative,"[[{'label': 'negative', 'score': 0.99975949525...",0.999759,negative
28,00000038-3b3b-5102-c7c2-0ab000000000,Apro l’app più volte a settimana e ogni volta ...,neutral,1.0,"[""problemi_accesso""]",0.0,1.0.0,2024-11-12,it,Apro l’app più volte a settimana e ogni volta ...,negative,negative,"[[{'label': 'negative', 'score': 0.99359303712...",0.993593,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3672,f408d06f-3a29-4c4f-85c4-d1231eafef9e,Correggo la valutazione data in precedenza per...,neutral,5.0,"[""problemi_sistema""]",0.0,1.0.0,2024-11-16,it,Correggo la valutazione data in precedenza per...,negative,negative,"[[{'label': 'negative', 'score': 0.99978524446...",0.999785,negative
3682,f5a79382-7a09-463d-83d6-bedc2668640b,Improvvisamente da ieri il mio saldo non è qua...,neutral,5.0,"[""problemi_pagamenti_generici""]",0.0,1.0.0,2024-11-19,it,Improvvisamente da ieri il mio saldo non è qua...,negative,negative,"[[{'label': 'negative', 'score': 0.99979835748...",0.999798,negative
3687,f68f611e-6380-4e6e-a2fc-68b98847aba3,Dopo l'ultimo aggiornamento è sparito il ricon...,neutral,2.0,"[""problemi_accesso""]",0.0,1.0.0,2025-01-19,it,Dopo l'ultimo aggiornamento è sparito il ricon...,negative,negative,"[[{'label': 'negative', 'score': 0.99827980995...",0.998280,negative
3696,f886e36a-faaf-423a-bc67-767174fcf788,Ottima App. Migliorerei la reportistica che fa...,neutral,5.0,"[""problemi_comunicazione""]",0.0,1.0.0,2024-10-16,it,Ottima App. Migliorerei la reportistica che fa...,neutral,positive,"[[{'label': 'positive', 'score': 0.99974793195...",0.999748,positive


In [38]:
# Mapping sentiment → numero
mapping = {"negative": -1, "neutral": 0, "positive": 1}

# Convert columns
dfNeu["v1"] = dfNeu["sentiment_pred"].map(mapping)
dfNeu["v2"] = dfNeu["sentiment_predMulti"].map(mapping)
dfNeu["v3"] = dfNeu["sentiment_predScoreLabel"].map(mapping)

# Composition score
dfNeu["sentiment_composition"] = dfNeu["v1"] + dfNeu["v2"] + dfNeu["v3"]

# Optional: final sentiment from numeric
def convert_numeric_to_label(val):
    if val > 0:
        return "positive"
    elif val <= 0:
        return "negative"


dfNeu["sentiment_final_numeric"] = dfNeu["sentiment_composition"].apply(convert_numeric_to_label)


In [41]:
display(dfNeu[['review', 'sentiment', 'sentiment_final_numeric']])

,review,sentiment,sentiment_final_numeric
6,"App perfetta, nessun altra banca ha questa imm...",neutral,negative
10,Buona app a volte non fluido l’accesso. App or...,neutral,negative
15,Appena aperto il conto Premium e spero di trov...,neutral,positive
18,"Salve, la funzionalità di tracciamento delle o...",neutral,negative
28,Apro l’app più volte a settimana e ogni volta ...,neutral,negative
...,...,...,...
3672,Correggo la valutazione data in precedenza per...,neutral,negative
3682,Improvvisamente da ieri il mio saldo non è qua...,neutral,negative
3687,Dopo l'ultimo aggiornamento è sparito il ricon...,neutral,negative
3696,Ottima App. Migliorerei la reportistica che fa...,neutral,positive


In [42]:
from google.colab import files
dfNeu.to_excel("dataFramNewNeutral.xlsx", index=True)
files.download("dataFramNewNeutral.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [45]:
import emoji
import re

# funzione per estrarre emoji da un testo
def extract_emojis(text):
    # emoji.emoji_list() returns a list of dicts, each with 'emoji' key
    return [item['emoji'] for item in emoji.emoji_list(str(text))]

# Filtra solo le review neutrali
df_neutral = df[df["sentiment"] == "neutral"].copy()

# Estrae le emoji per ogni review
df_neutral["emoji_list"] = df_neutral["review"].apply(lambda x: extract_emojis(str(x)))

# Conta quante emoji sono presenti per riga
df_neutral["emoji_count"] = df_neutral["emoji_list"].apply(len)

# Totale emoji nelle review neutrali
total_emojis = df_neutral["emoji_count"].sum()

print("Totale emoji nelle review neutrali:", total_emojis)



Totale emoji nelle review neutrali: 10


,review,emoji_list,emoji_count
6,"App perfetta, nessun altra banca ha questa imm...",[],0
10,Buona app a volte non fluido l’accesso. App or...,[],0
15,Appena aperto il conto Premium e spero di trov...,[],0
18,"Salve, la funzionalità di tracciamento delle o...",[],0
28,Apro l’app più volte a settimana e ogni volta ...,[],0


In [47]:
# Mostra le prime righe del risultato
display(df_neutral[["review", "emoji_list", "emoji_count"]])

,review,emoji_list,emoji_count
6,"App perfetta, nessun altra banca ha questa imm...",[],0
10,Buona app a volte non fluido l’accesso. App or...,[],0
15,Appena aperto il conto Premium e spero di trov...,[],0
18,"Salve, la funzionalità di tracciamento delle o...",[],0
28,Apro l’app più volte a settimana e ogni volta ...,[],0
...,...,...,...
3672,Correggo la valutazione data in precedenza per...,[],0
3682,Improvvisamente da ieri il mio saldo non è qua...,[],0
3687,Dopo l'ultimo aggiornamento è sparito il ricon...,[],0
3696,Ottima App. Migliorerei la reportistica che fa...,[],0




---



✔ FEEL-IT (pipeline)

✔ FEEL-IT (tokenizer + model)

✔ XLM-RoBERTa (tokenizer + model)